## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * estado_vacunacion_.csv , última versión del mismo con el acumulado de todos los informes publicados
  * estado_vacunacion_year-month-day.csv, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
fecha_informe = datetime(year=2021,month=1,day=7)
fecha_informe_str=fecha_informe.strftime('%Y%m%d')
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
datos_row = pd.read_excel(enlace)

In [5]:
datos_row

,Unnamed: 0,Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Fecha de la última vacuna registrada (2)
0,Andalucía,140295,53934,0.384433,2021-01-06
1,Aragón,23715,6186,0.260848,2021-01-06
2,Asturias,23720,14406,0.607336,2021-01-06
3,Baleares,12840,3190,0.248442,2021-01-05
4,Canarias,27660,7330,0.265004,2021-01-06
5,Cantabria,11850,1564,0.131983,2021-01-07
6,Castilla y Leon,51390,22103,0.430103,2021-01-06
7,Castilla La Mancha,35550,7666,0.215640,2021-01-06
8,Cataluña,120545,24515,0.203368,2021-01-06
9,C. Valenciana,61225,11677,0.190723,2021-01-07


..copia para tratarlo

In [6]:
datos_hoy = datos_row.copy()

renombramos columnas, añadimos columna con `fecha_de_informe` y ordenamos

In [7]:
datos_hoy.columns=['Comunidad','dosis_entregadas','dosis_administradas','porcentaje_administradas','fecha_vacuna']

In [8]:
datos_hoy['fecha_informe']=fecha_informe

In [9]:
datos_hoy=datos_hoy[['fecha_informe','Comunidad','fecha_vacuna','dosis_entregadas','dosis_administradas','porcentaje_administradas']]

eliminamos la fila Totales

In [10]:
datos_hoy = datos_hoy.drop(datos_hoy.index[-1])

In [11]:
datos_hoy

,fecha_informe,Comunidad,fecha_vacuna,dosis_entregadas,dosis_administradas,porcentaje_administradas
0,2021-01-07,Andalucía,2021-01-06,140295,53934,0.384433
1,2021-01-07,Aragón,2021-01-06,23715,6186,0.260848
2,2021-01-07,Asturias,2021-01-06,23720,14406,0.607336
3,2021-01-07,Baleares,2021-01-05,12840,3190,0.248442
4,2021-01-07,Canarias,2021-01-06,27660,7330,0.265004
5,2021-01-07,Cantabria,2021-01-07,11850,1564,0.131983
6,2021-01-07,Castilla y Leon,2021-01-06,51390,22103,0.430103
7,2021-01-07,Castilla La Mancha,2021-01-06,35550,7666,0.215640
8,2021-01-07,Cataluña,2021-01-06,120545,24515,0.203368
9,2021-01-07,C. Valenciana,2021-01-07,61225,11677,0.190723


guardamos el fichero de hoy en el directorio csv

In [18]:
#datos_hoy.to_csv(nombre_fichero_acumulado+'.csv',index = False)

In [19]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [30]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [31]:
datos_acumulados

,fecha_informe,Comunidad,fecha_vacuna,dosis_entregadas,dosis_administradas,porcentaje_administradas
0,2021-01-07,Andalucía,2021-01-06,140295,53934,0.384433
1,2021-01-07,Aragón,2021-01-06,23715,6186,0.260848
2,2021-01-07,Asturias,2021-01-06,23720,14406,0.607336
3,2021-01-07,Baleares,2021-01-05,12840,3190,0.248442
4,2021-01-07,Canarias,2021-01-06,27660,7330,0.265004
5,2021-01-07,Cantabria,2021-01-07,11850,1564,0.131983
6,2021-01-07,Castilla y Leon,2021-01-06,51390,22103,0.430103
7,2021-01-07,Castilla La Mancha,2021-01-06,35550,7666,0.215640
8,2021-01-07,Cataluña,2021-01-06,120545,24515,0.203368
9,2021-01-07,C. Valenciana,2021-01-07,61225,11677,0.190723


concatenamos...

In [23]:
datos_acumulados = datos_acumulados.append(datos_hoy, ignore_index=True)

In [28]:
#aseguramos formato fecha para evitar inconsistencia en el merge
datos_acumulados['fecha_informe'] = pd.to_datetime(datos_acumulados['fecha_informe'])
datos_acumulados['fecha_vacuna'] = pd.to_datetime(datos_acumulados['fecha_vacuna'])

In [32]:
datos_acumulados

,fecha_informe,Comunidad,fecha_vacuna,dosis_entregadas,dosis_administradas,porcentaje_administradas
0,2021-01-07,Andalucía,2021-01-06,140295,53934,0.384433
1,2021-01-07,Aragón,2021-01-06,23715,6186,0.260848
2,2021-01-07,Asturias,2021-01-06,23720,14406,0.607336
3,2021-01-07,Baleares,2021-01-05,12840,3190,0.248442
4,2021-01-07,Canarias,2021-01-06,27660,7330,0.265004
5,2021-01-07,Cantabria,2021-01-07,11850,1564,0.131983
6,2021-01-07,Castilla y Leon,2021-01-06,51390,22103,0.430103
7,2021-01-07,Castilla La Mancha,2021-01-06,35550,7666,0.215640
8,2021-01-07,Cataluña,2021-01-06,120545,24515,0.203368
9,2021-01-07,C. Valenciana,2021-01-07,61225,11677,0.190723


y volvemos a subirlo..

In [ ]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index_label=False)